https://towardsdatascience.com/how-to-fine-tune-gpt-2-for-text-generation-ae2ea53bc272

In [1]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import re
import os

c:\Users\laudi\anaconda3\envs\work\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PATH_FIND = re.compile(r".*repository.")
rep_path = re.search(PATH_FIND,os.getcwd()).group()

In [3]:
torch.cuda.is_available()

True

In [4]:


### Prepare data
df = pd.read_csv(rep_path+"\\99 generated csv\\ch 2\\raw_all_component.csv ")
# lyrics = pd.read_csv('lyrics-data.csv')
# lyrics = lyrics[lyrics['Idiom']=='ENGLISH']

#Only keep popular artists, with genre Rock/Pop and popularity high enough
# artists = pd.read_csv('artists-data.csv')
# artists = artists[(artists['Genre'].isin(['Rock'])) & (artists['Popularity']>5)]
# df = lyrics.merge(artists[['Artist', 'Genre', 'Link']], left_on='ALink', right_on='Link', how='inner')
# df = df.drop(columns=['ALink','SLink','Idiom','Link'])

#Drop the songs with lyrics too long (after more than 1024 tokens, does not work)
# df = df[df['Lyric'].apply(lambda x: len(x.split(' ')) < 350)]

# #Create a very small test set to compare generated text with the reality
test_set = df.sample(n = 200, random_state=1998)
df = df.loc[~df.index.isin(test_set.index)]

#Reset the indexes
test_set = test_set.reset_index()
df = df.reset_index()

#For the test set only, keep last 20 words in a new column, then remove them from original column
test_set['True_end_abstract'] = test_set['abstract'].str.split().str[-20:].apply(' '.join)
test_set['Abstract'] = test_set['abstract'].str.split().str[:-20].apply(' '.join)

In [3]:
os.getcwd()

'c:\\Users\\laudi\\OneDrive\\Desktop\\Tesi_workspace\\repository\\3 preparation'

In [3]:
# PATH_FIND = re.compile(".*repository.")
# rep_path = re.search(PATH_FIND,os.getcwd()).group()
os.chdir(r"C:\Users\laudi\OneDrive\Desktop\Tesi_workspace\repository\3 preparation")

if "python training" not in os.listdir():
    os.makedirs(r"C:\Users\laudi\OneDrive\Desktop\Tesi_workspace\repository\3 preparation\python training")


os.chdir("python training")
print(os.getcwd())
# ### Prepare data
# df = pd.read_csv(rep_path+"\\99 generated csv\\ch 2\\raw_all_component.csv ")
df = pd.read_csv(r"C:\Users\laudi\OneDrive\Desktop\Tesi_workspace\repository\99 generated csv\ch 2\raw_all_component.csv ")

# #Create a very small test set to compare generated text with the reality
isfirst = input("Is it the first iteration? [Y/N]\n>>> ")
while True:
    if isfirst == "Y":
        test_set = df.sample(n = 40, random_state=1998)
        df = df.loc[~df.index.isin(test_set.index)]
        test_set = test_set.reset_index()
        df = df.reset_index()
        if "csv" not in os.listdir():
            os.makedirs("csv")
        test_set.to_csv(r"..//csv//test_set_gpt2.csv", index = False)
        df.to_csv(r"..//csv//training_set_gpt2.csv", index = False)
        break
    elif isfirst == "N":
        test_set = pd.read_csv(r"csv\test_set_gpt2.csv")
        df = pd.read_csv(r"csv\training_set_gpt2.csv")
        break
    else:
        isfirst = input("Is it the first iteration? [Y/N]\n>>> ")

    

C:\Users\laudi\OneDrive\Desktop\Tesi_workspace\repository\3 preparation\python training


In [6]:
range_training = input("select your range[1,2,3,4,5,6,7,8,9,10]:\n>>>")
if range_training == "1":
    df = df.iloc[:1000, :] 
elif range_training == "2":
    df = df.iloc[1000:2000, :]
elif range_training == "3":
    df = df.iloc[2000:3000, :]
elif range_training == "4":
    df = df.iloc[3000:4000, :]
elif range_training == "5":
    df = df.iloc[4000:5000, :]
elif range_training == "6":
    df = df.iloc[5000:6000, :]
elif range_training == "7":
    df = df.iloc[6000:7000, :]
elif range_training == "8":
    df = df.iloc[7000:8000, :]
elif range_training == "9":
    df = df.iloc[8000:9000, :]

#Reset the indexes


#For the test set only, keep last 20 words in a new column, then remove them from original column
test_set['True_end_abstract'] = test_set['abstract'].str.split().str[-20:].apply(' '.join)
#test_set['Abstract'] = test_set['abstract'].str.split().str[:-20].apply(' '.join)

#### Useful class for Abstract

class Abstract(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.abstract = []
        c = 0 
        for row in df['abstract']:
            self.abstract.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))      
            if c%2000 == 0:
                print(f"iter n {c}")
            c+=1
        if truncate:
            self.abstract = self.abstract[:2000] #it was 20000
        self.abstract_count = len(self.abstract)
        
    def __len__(self):
        return self.abstract_count

    def __getitem__(self, item):
        return self.abstract[item]
isfirst = input("Is it the first model? [Y/N]\n>>> ")
while True:
    if isfirst == "Y":
        dataset = Abstract(df['abstract'], truncate=False, gpt2_type="gpt2")
        #Get the tokenizer and model
        tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        model = GPT2LMHeadModel.from_pretrained('gpt2')
        break
    elif isfirst == "N":
        last = os.listdir("pretrained")[-1]
        dataset = Abstract(df['abstract'], truncate=False, gpt2_type="gpt2")
        #Get the tokenizer and model
        tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        model = GPT2LMHeadModel.from_pretrained(r"pretrained\{last_}".format(last_=last))
        break
    else:
        isfirst = input("Is it the first model? [Y/N]\n>>> ")
#get the class

iter n 0


In [5]:
class Abstract(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.abstract = []
        c = 0 
        for row in df['abstract']:
            self.abstract.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))      
            if c%1000 == 0:
                print(f"iter n {c}")
            c+=1
        if truncate:
            self.abstract = self.abstract[:20000]
        self.abstract_count = len(self.abstract)
        
    def __len__(self):
        return self.abstract_count

    def __getitem__(self, item):
        return self.abstract[item]
    


In [6]:
dataset = Abstract(df['abstract'], truncate=True, gpt2_type="gpt2") 

iter n 0
iter n 1000
iter n 2000
iter n 3000
iter n 4000
iter n 5000
iter n 6000
iter n 7000
iter n 8000
iter n 9000
iter n 10000
iter n 11000
iter n 12000
iter n 13000
iter n 14000
iter n 15000
iter n 16000
iter n 17000
iter n 18000
iter n 19000
iter n 20000
iter n 21000
iter n 22000
iter n 23000
iter n 24000
iter n 25000
iter n 26000
iter n 27000
iter n 28000
iter n 29000
iter n 30000
iter n 31000
iter n 32000
iter n 33000
iter n 34000
iter n 35000
iter n 36000
iter n 37000
iter n 38000
iter n 39000
iter n 40000
iter n 41000
iter n 42000
iter n 43000
iter n 44000
iter n 45000
iter n 46000
iter n 47000
iter n 48000
iter n 49000


In [12]:
model = GPT2LMHeadModel.from_pretrained(r"pretrained/pretrained-2023-01-22/")

In [3]:
#Get the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [8]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=5, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,save_model = False
):
    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
        if save_model:
            model.save_pretrained(f"pretrained//pretrained-{str(dt.datetime.now())[:10]}")
    return model

In [6]:
import datetime as dt
f"{str(dt.datetime.now())[:10]}-h{str(dt.datetime.now())[11:13]}-m{str(dt.datetime.now())[14:16]}"

'2023-01-24-h14-m41'

In [ ]:
model = train(dataset, model, tokenizer, batch_size=1, epochs=3, save_model_on_epoch = True)

Training epoch 0
0


3118it [03:58, 13.44it/s]

In [16]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
                output_list = list(generated.squeeze().numpy())
                output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
                generated_list.append(output_text)
                
    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
    generated_abstract = []
    for i in range(len(test_data)):
        x = generate(model.to('cpu'), tokenizer, test_data['Abstract'][i], entry_count=1)
        generated_abstract.append(x)
    return generated_abstract

In [20]:
#Run the functions to generate the lyrics
generated_abstarct = text_generation(test_set)

  0%|                                                                                            | 0/1 [00:03<?, ?it/s]

KeyboardInterrupt



In [19]:
#Loop to keep only generated text and add it as a new column in the dataframe
my_generations=[]

for i in range(len(generated_abstract)):
    a = test_set['Abstract'][i].split()[-30:] #Get the matching string we want (30 words)
    b = ' '.join(a)
    c = ' '.join(generated_abstract[i]) #Get all that comes after the matching string
    my_generations.append(c.split(b)[-1])

test_set['Generated_abstract'] = my_generations


#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(test_set)):
    to_remove = test_set['Generated_abstract'][i].split('.')[-1]
    final.append(test_set['Generated_abstract'][i].replace(to_remove,''))

test_set['Generated_abstract'] = final

NameError: name 'generated_abstract' is not defined

In [ ]:

#Using BLEU score to compare the real sentences with the generated ones
import statistics
from nltk.translate.bleu_score import sentence_bleu

scores=[]

for i in range(len(test_set)):
    reference = test_set['True_end_lyrics'][i]
    candidate = test_set['Generated_lyrics'][i]
    scores.append(sentence_bleu(reference, candidate))

statistics.mean(scores)